## IMDClient package architecture

## TODO:

a context manager?
better error messages
remove unprofessional log messages
add docs for some of the messages (timeout at least)
make a nice diagram

In [ ]:
# ./run.sh 

import imdclient

client = imdclient.IMDClient("localhost", 8889, n_atoms=100)

info = client.get_imdsessioninfo()
## TODO: show what's in dict
## design principle: reduce redundancy in data entry

client.stop()

In [ ]:
client = imdclient.IMDClient("localhost", 8889, n_atoms=100)

## Design principle: simplest possible API so anyone can use it
## Even if you hate mdanalysis, you can use this

while True:
    try:
        frame = client.get_imdframe()
    except EOFError:
        break

client.stop()

## Options

In [ ]:
client = imdclient.IMDClient("localhost", 8889, n_atoms=100, timeout=10)

while True:
    try:
        ## Add a little wait time widget
        frame = client.get_imdframe()
    except EOFError:
        break

client.stop()

## The buffer size is optional, pausing is automatic

In [ ]:
# ./run.sh

import time
import logging

logger = logging.getLogger("imdclient.IMDClient")
# Modify to use 
# file_handler = logging.FileHandler("gromacs_test.log")
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
# file_handler.setFormatter(formatter)
# logger.addHandler(file_handler)
# logger.setLevel(logging.DEBUG)

## Show the logs of the client stopping automatically

client = imdclient.IMDClient("localhost", 8889, n_atoms=100, buffer_size=100)

## Reader wraps client, handles its limitations

In [ ]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda

u = mda.Universe(top, "imd://localhost:8889")

for ts in u.trajectory[:]:
    pass

try:
    for ts in u.trajectory[:]:
        pass
except RuntimeError:
    print("A stream is not a file!")

## Stepping is allowed, but specifying an end frame isn't

In [1]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda

u = mda.Universe(top, "imd://localhost:8889")

for ts in u.trajectory[::10]:
    pass

NameError: name 'top' is not defined

## The client works out of the box with MDAnalysis analysis classes

In [ ]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

u = mda.Universe(top, "imd://localhost:8889")

imd_rmsf = RMSF(u.atoms).run()

## Running multiple analysis classes on the same stream

We showed you examples earlier that look like this:

## But you can also do this:

In [ ]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

r1 = RMSF(u.atoms)
r2 = RMSF(u.atoms)
imdclient.StackableAnalysis(u.trajectory, [r1, r2]).run()